In [ ]:
# There are many ways in doing these, but I have kept only finalized codes here for easier references.
# Just run every function column-bu-column and the last calling command in the script will help to generate the end result
# If one wanted to see a full version of my trial with comments/references, please email me at chooboonlinn@gmail.com
# I can send you a messy but complete version by email upon request

# Do note that I declare the font param for matplotlib multiple times as assurance because previously when I was running
# these, somehow it goes hirewire after a long run, so better be safe than sorry

In [ ]:
%run ./CommonFunctions_OthersGeneral.ipynb  ## This is important for library import, getting the path and runlist

In [ ]:
import matplotlib as mpl
from math import degrees
import matplotlib.font_manager as font_manager
from matplotlib.ticker import FormatStrFormatter
from keras.models import load_model

mpl.rcParams.update({'font.size': 20})
#from pylab import *

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" # nothing = CPU, 0 = GeForce GTX 1660, 1 = Quadro K1200
## Although Quadro has became 0 and GTX became 1 (interchanged the ID) in 'NVIDIA X Server Settings' after I disabled GTX in X Server Display Configuration
## This does not affect the ID in the system (can see nvidia-smi), as tentatively, the fastest GPU is defined as 0
## https://shawnliu.me/post/nvidia-gpu-id-enumeration-in-linux/
## https://stackoverflow.com/questions/13781738/how-does-cuda-assign-device-ids-to-gpus

## Lines below must be performed for Ubuntu 18.04.3
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
## These steps have previously been performed, so re-run is not necessary under normal circumstances

## To construct necessary tables from available data ##
#%run ./Step1_BuildingTables.ipynb

## To do position plotting to check on topography and selectively checking on position from 8 different angles ##
#%run ./Step2_XY_Checking.ipynb 

#K.clear_session()

In [ ]:
# Files name for current run, please alter accordingly

dense = 128
lr_rate = 0.00001
grid_division = 25
samplingsize = 8
epochs=100
test_runNum = '1.0.1.' + str(dense) + '-NoTraining260520'#'-100epochs-lr=' + str(lr_rate)
#lrr = '{:.5f}'.format(0.00001) #'{:f}'.format(1e-05).rstrip('0')
#test_runNum = '1.0.1.' + str(dense) + '-100epochs-lr=' + str(lrr)
print(test_runNum)

In [ ]:
# Step 3

grid_division = 25
samplingsize = 8
##batchsize = samplingsize
# mode defined in the calling of function
# runlist is adapted from %run ./CommonFunctions_OthersGeneral.ipynb
# chosen_angle is defined inside the function: training = ['45°','135°','225°','315°'] and testing = ['0°/360°','90°','180°','270°']

%run ./Step3_ImagePreprocessing_DataGenerator-GroupNormalization-fullRGBmean-FANCY.ipynb

## Normlization via Group Mean & Std of Image Shape
%time allMean_data,allStd_data = group_mean()

%time training_data = datagenerator_prep1_extractdata(allMean_data,allStd_data,runlist,grid_division,mode='training')
%time testing_data = datagenerator_prep1_extractdata(allMean_data,allStd_data,runlist,grid_division,mode='testing')
#%time training_data = datagenerator_prep1_extractdata_Shuffle_250520(allMean_data,allStd_data,runlist,grid_division,mode='training') # Model Reliability Testing
#%time testing_data = datagenerator_prep1_extractdata_Shuffle_250520(allMean_data,allStd_data,runlist,grid_division,mode='testing') # Model Reliability Testing

normalized_illustration_randomcheck(training_data,samplingsize,mode='training')
normalized_illustration_randomcheck(testing_data,samplingsize,mode='testing')

%time training_generator = datagenerator(training_data,samplingsize,angles='no')
%time testing_generator = datagenerator(testing_data,samplingsize,angles='no')

## Model Reliability Testing: With manual percentage split ##
#%time training_data,testing_data = dataProcessAll_PercentageTest_Step1(runlist,grid_division,train_percentage=0.2)
#%time training_generator = datagenerator_PercentageTest_Step2(training_data, samplingsize, angles='no')
#%time testing_generator = datagenerator_PercentageTest_Step2(testing_data, samplingsize, angles='no')


In [ ]:
%run ./Step4_CNN_Regression.ipynb
model = create_cnn_functional_model(angles='no',num_layers=4)

# Trying on "reinitialize the weights to be randomly distributed"
#model = create_cnn_functional_model_WeightsPerturb(angles='no',num_layers=4,randomise='yes',randomiseMethod='3')
#model = create_cnn_functional_model_WeightsPerturb(angles='no',num_layers=4,randomise='no',randomiseMethod='')

In [ ]:
def trainTheModel(lr,factor,patience,steps,epochs):#,min_lr):
    if lr == 'yes':
        from keras.callbacks import CSVLogger
        from keras.callbacks import ReduceLROnPlateau

        filepath = dir23 + "/" + test_runNum + "-fullRGBmean-FANCY_lr=" + lr + ".log"
        csv_logger = CSVLogger(filepath, separator=',', append=False)

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience)#, min_lr=min_lr)
        
        model.fit_generator(training_generator,validation_data=testing_generator,steps_per_epoch=steps,validation_steps=steps,epochs=epochs,callbacks=[reduce_lr,csv_logger],use_multiprocessing=True)

    elif lr == 'no':
        from keras.callbacks import CSVLogger

        filepath = dir23 + "/" + test_runNum + "-fullRGBmean-FANCY_lr=" + lr + ".log"
        csv_logger = CSVLogger(filepath, separator=',', append=False)
    
        model.fit_generator(training_generator,validation_data=testing_generator,steps_per_epoch=steps,validation_steps=steps,epochs=epochs,callbacks=[csv_logger],use_multiprocessing=True)


In [ ]:
trainTheModel(lr='no',factor='',patience='',steps=1000,epochs=100)
#trainTheModel(lr='yes',factor=0.1,patience=5,steps=1000,epochs=100)#, min_lr=0.00001)

In [ ]:
# This would save all the main trained parameters such as weights too instead of just the model's performance
filepath = dir23 + "/evaluateCallbacks/" + test_runNum + "-fullRGBmean-FANCY_lr=no.h5" #+ lr + ".h5"
model.save(filepath)

# To load this saved model:
#from keras.models import load_model
#model = load_model(filepath)

In [ ]:
def plot_history(angles,lr):
    plt.clf()
    filepath = dir23 + "/" + test_runNum + "-fullRGBmean-FANCY_lr=" + lr + ".log"
    logdata = pd.read_csv(filepath)
    
    if angles == 'yes':
        # Plot training & validation accuracy values
        plt.plot(logdata.loss)
        plt.plot(logdata.val_loss)
        plt.title('Model Loss',fontsize=25, fontweight='bold')
        plt.ylabel('Loss',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'a-Model_Loss.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        plt.plot(logdata.output1_xy_loss)
        plt.plot(logdata.val_output1_xy_loss)
        plt.title('XY Loss',fontsize=25, fontweight='bold')
        plt.ylabel('Loss',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'b-XY_Loss.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        plt.plot(logdata.output2_angles_loss)
        plt.plot(logdata.val_output2_angles_loss)
        plt.title('Angles Loss',fontsize=25, fontweight='bold')
        plt.ylabel('Angles Loss',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'c-Angles_Loss.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        # Plot training & validation loss values
        plt.plot(logdata.output1_xy_mean_squared_error)
        plt.plot(logdata.val_output1_xy_mean_squared_error)
        plt.title('XY MSE',fontsize=25, fontweight='bold')
        plt.ylabel('MSE',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'd-XY_MSE.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        plt.plot(logdata.output2_angles_mean_squared_error)
        plt.plot(logdata.val_output2_angles_mean_squared_error)
        plt.title('Angles MSE',fontsize=25, fontweight='bold')
        plt.ylabel('MSE',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'e-Angles_MSE.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        # Plot training & validation accuracy values
        plt.plot(logdata.output1_xy_acc)
        plt.plot(logdata.val_output1_xy_acc)
        plt.title('XY Accuracy',fontsize=25, fontweight='bold')
        plt.ylabel('Accuracy',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'f-XY_Accuracy.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

        plt.plot(logdata.output2_angles_acc)
        plt.plot(logdata.val_output2_angles_acc)
        plt.title('Angles Accuracy',fontsize=25, fontweight='bold')
        plt.ylabel('Accuracy',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'g-Angles_Accuracy.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

    elif angles == 'no':
        # Plot training & validation accuracy values
        plt.plot(logdata.loss)
        plt.plot(logdata.val_loss)
        plt.title('Model Loss',fontsize=25, fontweight='bold')
        plt.ylabel('Loss',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'a-Model_Loss.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
        
        # Plot training & validation loss values
        plt.plot(logdata.mean_squared_error)
        plt.plot(logdata.val_mean_squared_error)
        plt.title('Model MSE',fontsize=25, fontweight='bold')
        plt.ylabel('MSE',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'b-Model_MSE.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
        
        # Plot training & validation accuracy values
        plt.plot(logdata.acc)
        plt.plot(logdata.val_acc)
        plt.title('Model Accuracy',fontsize=25, fontweight='bold')
        plt.ylabel('Accuracy',fontsize=22)
        plt.xlabel('Epoch',fontsize=22)
        lgd = plt.legend(['Train', 'Test'], loc='upper right',fontsize=22, bbox_to_anchor=(1.5, 1.05))
        plt.grid('on')        
        fig = plt.gcf()
        fig.set_size_inches(6.5,4.5)
        plt.show()
        fig.savefig(dir16 + '/' + test_runNum + 'c-Model_Accuracy.png', bbox_extra_artists=(lgd,), bbox_inches='tight') #https://stackoverflow.com/questions/10101700/moving-matplotlib-legend-outside-of-the-axis-makes-it-cutoff-by-the-figure-box
        

In [ ]:
def activationModelSelection_Angles(angles,mode,times):
    mpl.rcParams.update({'font.size': 20})
   
    def real_pred_error(angles):
        x_error = []
        y_error = []
        euclideanDist_error = []

        if angles == 'no':
            for i in range(0, real_xy.shape[0]):
                x = real_xy[i][0]-pred_xy[i][0] #similar to real_xy[i,0]-pred_xy[i,0]
                x_error.append(x)
                y = real_xy[i][1]-pred_xy[i][1] #similar to real_xy[i,1]-pred_xy[i,1]
                y_error.append(y)
                dist = np.linalg.norm(real_xy[i]-pred_xy[i])
                euclideanDist_error.append(dist)
            return x_error, y_error, euclideanDist_error

        elif angles == 'yes':
            angles_error = []

            for i in range(0, real_xy.shape[0]):
                x = real_xy[i][0]-pred_xy[i][0] #similar to real_xy[i,0]-pred_xy[i,0]
                x_error.append(x)
                y = real_xy[i][1]-pred_xy[i][1] #similar to real_xy[i,1]-pred_xy[i,1]
                y_error.append(y)
                dist = np.linalg.norm(real_xy[i]-pred_xy[i])
                euclideanDist_error.append(dist)
                angles = real_angles-pred_angles
                angles_error.append(angles)
            return x_error, y_error, euclideanDist_error, angles_error
            
    def cart2pol(x, y): # https://github.com/numpy/numpy/issues/5228 # https://www.w3resource.com/python-exercises/numpy/python-numpy-random-exercise-14.php
        theta = np.arctan2(y, x)
        rho = np.hypot(x, y)
        return theta, rho

    def matplotlib_polarplot(theta, rho):
        degree = []
        #degree_ = degrees(theta) # if use from pylab import *
        degree_ = np.degrees(theta)
        #degree_ = (np.degrees(theta)).flatten() # not needed unless the real_xy or pred_xy was not reshape beforehand
        for i in degree_:
            if i < 0: #if any(i) < 0: # if not flatten
                i = 360+i
            degree.append(i)

        fig, axes = plt.subplots(1,2, figsize=(20,15), tight_layout=True)
        #axes = axes.flatten() # Not needed unless the real_xy or pred_xy was not reshape beforehand

        for idx, ax in enumerate(axes):
            if idx == 0:
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax.set_frame_on(False)
                ax = fig.add_subplot(121, polar=True) #ax = plt.subplot(111, projection='polar') # Subplot with 1 row and 1 column at the index 1
                ax.scatter(theta, rho, color='indigo',alpha=0.4)#,label='Degree')
                ax.grid(True, linewidth=1)
                ax.set_theta_zero_location("N")
                ax.set_theta_direction(-1)
                ax.set_theta_offset(pi/2.0)
                #ax.legend()
                ax.set_title('Polar Scattered Plot - XY Error', va='bottom', pad=5, fontsize=25, fontweight='bold')        
            elif idx == 1:
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax.set_frame_on(False)
                ax = fig.add_subplot(122)
                ax.set_title('Histogram - XY Error (Degree)', pad=5, fontsize=25, fontweight='bold')
                ax.hist(degree, bins=36, alpha=0.5, label='theta radians into degree',color='orchid')
                #major_ticks = np.arange(0, 1, 0.1)
                minor_ticks = np.arange(0, 360, 10)
                #ax.set_yticks(major_ticks)
                ax.set_xticks(minor_ticks, minor=True)
                ax.grid(which='both', alpha=0.5)
                ax.set_xlabel('Degree', fontsize=22, fontweight='bold')
                ax.set_ylabel('Count', fontsize=22, fontweight='bold')


        GeneralTitle = 'XY Error Computation'
        fig.suptitle((GeneralTitle + "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"), fontsize=50, fontweight='bold', ha='center', va='top', y=1.08)
        #fig.tight_layout()

    # Combining functions above into one
    def plotting_XY_angles_error(angles,mode):
        if angles == 'no':
            x_error, y_error, euclideanDist_error = real_pred_error(angles)
            theta, rho = cart2pol(x_error, y_error)
            polarplot = matplotlib_polarplot(theta, rho)
            plt.savefig(dir21 + '/' + test_runNum + '-' + mode + '_xyDiff_polarPlot.png')

        elif angles == 'yes':
            x_error, y_error, euclideanDist_error = real_pred_error(angles)
            theta, rho = cart2pol(x_error, y_error)
            polarplot = matplotlib_polarplot(theta, rho)
            plt.savefig(dir21 + '/' + test_runNum + '-' + mode + '_xyDiff_polarPlot.png')
            print("Sorry, only XY error plot has been completed at the moment. Angles error plot is subject to trial in the future.")
    
    #################################
        
    if angles == 'yes':  
        real_xy, pred_xy, real_angles, pred_angles, real_img = [], [], [], [], []
    
        if mode == 'training' or mode == 'learning':        
            for i in range(0, times):
                (X,y) = next(training_generator) #(next(training_generator))[0] = images,(next(training_generator))[1][0] = xy,(next(training_generator))[1][1] = angles
                y_pred = model.predict(X)
                real_xy.append(y[0])
                pred_xy.append(y_pred[0]) #because we just wanted xy and not angles
                real_angles.append(y[1])
                pred_angles.append(y_pred[1]) #because we just wanted xy and not angles
                real_img.append(X)
            real_xy, pred_xy, real_angles, pred_angles, real_img = np.array(real_xy), np.array(pred_xy), np.array(real_angles), np.array(pred_angles), np.array(real_img)
                        
            real_xy = np.reshape(real_xy, (real_xy.shape[0] * real_xy.shape[1], -1))
            pred_xy = np.reshape(pred_xy, (pred_xy.shape[0] * pred_xy.shape[1], -1))
            real_angles = np.reshape(real_angles, (real_angles.shape[0] * real_angles.shape[1], -1))
            pred_angles = np.reshape(pred_angles, (pred_angles.shape[0] * pred_angles.shape[1], -1))       
            real_img = np.reshape(real_img, (real_img.shape[0] * real_img.shape[1], 108, 192, 3))
                          
            plotting_XY_angles_error(angles=angles,mode=mode)
            
        elif mode == 'testing':
            for i in range(0, times):
                (X,y) = next(testing_generator) #(next(testing_generator))[0] = images,(next(testing_generator))[1][0] = xy,(next(testing_generator))[1][1] = angles
                y_pred = model.predict(X)
                real_xy.append(y[0])
                pred_xy.append(y_pred[0]) #because we just wanted xy and not angles
                real_angles.append(y[1])
                pred_angles.append(y_pred[1]) #because we just wanted xy and not angles
                real_img.append(X)
            real_xy, pred_xy, real_angles, pred_angles, real_img = np.array(real_xy), np.array(pred_xy), np.array(real_angles), np.array(pred_angles), np.array(real_img)
             
            real_xy = np.reshape(real_xy, (real_xy.shape[0] * real_xy.shape[1], -1))
            pred_xy = np.reshape(pred_xy, (pred_xy.shape[0] * pred_xy.shape[1], -1))
            real_angles = np.reshape(real_angles, (real_angles.shape[0] * real_angles.shape[1], -1))
            pred_angles = np.reshape(pred_angles, (pred_angles.shape[0] * pred_angles.shape[1], -1))        
            real_img = np.reshape(real_img, (real_img.shape[0] * real_img.shape[1], 108, 192, 3))

            plotting_XY_angles_error(angles=angles,mode=mode)

        else: ## In case user forget to define the mode, the program will be halted
            raise ValueError('Please select mode of paradigm of either <training> or <testing>')
            pass

        return real_xy, pred_xy, real_angles, pred_angles, real_img #,x_error, y_error, euclideanDist_error, angles_error
        
    elif angles == 'no':
        real_xy, pred_xy, real_img = [], [], []
    
        if mode == 'training' or mode == 'learning':        
            for i in range(0, times):
                (X,y) = next(training_generator) #(next(training_generator))[0] = images,(next(training_generator))[1][0] = xy,(next(training_generator))[1][1] = angles
                y_pred = model.predict(X)
                real_xy.append(y)
                pred_xy.append(y_pred) #because we just wanted xy and not angles
                real_img.append(X)
            real_xy, pred_xy, real_img = np.array(real_xy), np.array(pred_xy), np.array(real_img)
            
            real_xy = np.reshape(real_xy, (real_xy.shape[0] * real_xy.shape[1], -1))
            pred_xy = np.reshape(pred_xy, (pred_xy.shape[0] * pred_xy.shape[1], -1))
            real_img = np.reshape(real_img, (real_img.shape[0] * real_img.shape[1], 108, 192, 3))

            plotting_XY_angles_error(angles=angles,mode=mode)

        elif mode == 'testing':
            for i in range(0, times):
                (X,y) = next(testing_generator)
                y_pred = model.predict(X)
                real_xy.append(y)
                pred_xy.append(y_pred)
                real_img.append(X)
            real_xy, pred_xy, real_img = np.array(real_xy), np.array(pred_xy), np.array(real_img)
            
            real_xy = np.reshape(real_xy, (real_xy.shape[0] * real_xy.shape[1], -1))
            pred_xy = np.reshape(pred_xy, (pred_xy.shape[0] * pred_xy.shape[1], -1))
            real_img = np.reshape(real_img, (real_img.shape[0] * real_img.shape[1], 108, 192, 3))
                          
            plotting_XY_angles_error(angles=angles,mode=mode)

        else: ## In case user forget to define the mode, the program will be halted
            raise ValueError('Please select mode of paradigm of either <training> or <testing>')
            pass

        return real_xy, pred_xy, real_img #,x_error, y_error, euclideanDist_error

In [ ]:
def AnglesPredictionPlot(angles,mode):
    
    # Prediction vs Real Map
    def realPredPlot(real_xy, pred_xy, real_img, mode): # cont'd from return values above
        mpl.rcParams.update({'font.size': 20})
        plt.plot(real_xy[:,0], real_xy[:,1], 'bo')
        plt.plot(pred_xy[:,0], pred_xy[:,1], 'ro')
        plt.plot([real_xy[:,0], pred_xy[:,0]], [real_xy[:,1], pred_xy[:,1]], 'k-', alpha=0.25) # alpha is the transparency rating of the line
        plt.title((f'{mode} generator'),fontsize=25, fontweight='bold')
        lgd = plt.legend(['Actual XY', 'Predicted XY'], loc='upper right',fontsize=15, bbox_to_anchor=(1.5, 1.05))
        fig = plt.gcf()
        plt.show()
        print(f'The shape for real_img is {(real_img.shape)}')
        fig.savefig(dir17 + '/' + test_runNum + '-' + mode + '_xy_realPredPlot.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

    # Angles & Activations
       # Declared above, but in-function (function between function), and altered plot, so I'm declaring again
    def matplotlib_polarplot(theta, rho): # theta is radian(degree), rho is the neuron activation
        ax = plt.subplot(111, projection='polar')
        ax.plot(theta, rho)
        ax.grid(True, linewidth=1)

    def angles_distribution_check(mode,realOrPred):
        import itertools
        import matplotlib.pyplot as plt

        if realOrPred == 'real':       
            plt.plot(real_angles, hidden_act)
            plt.show()
            plt.savefig(dir17 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_angles_linearPlot.png')
            polarplot = matplotlib_polarplot(real_angles, hidden_act)
            plt.savefig(dir17 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_angles_polarPlot.png')

        elif realOrPred == 'pred':
            plt.plot(pred_angles, hidden_act)
            plt.show()
            plt.savefig(dir17 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_angles_linearPlot.png')
            polarplot = matplotlib_polarplot(pred_angles, hidden_act)
            plt.savefig(dir17 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_angles_polarPlot.png')
    
    ##########
    
    if angles == 'yes':
        realPredPlot(real_xy=real_xy, pred_xy=pred_xy, real_img=real_img, mode=mode)
        angles_distribution_check(mode=mode,realOrPred='real')
        angles_distribution_check(mode=mode,realOrPred='pred')
    
    elif angles == 'no':
        realPredPlot(real_xy=real_xy, pred_xy=pred_xy, real_img=real_img, mode=mode)

In [ ]:
# Adapted from Markus's codes

from scipy.ndimage import gaussian_filter

def generate_ratemap_heatmap_from_activations(neuron_activation, positions, num_bins=25, smoothing=1, max_pos=None):
    #"""
    #Generates ratemap for given neuron_activations and positions. Every value in positions has a corresponding neuron_activation
    #"""
    # If not square make bins square and use unequal amount of bins
    if max_pos is None:
        max_pos = np.max(positions, axis=0)
    #if np.argmax(max_pos) == 0:
    #    square_bins = [num_bins, int(num_bins * (max_pos[0] / max_pos[1]))]
    #else:
    #    square_bins = [int(num_bins * (max_pos[1] / max_pos[0])), num_bins]
    
    square_bins = [num_bins,num_bins] # edited on 030420 starting Run #2 to make sure it is (25,25) bins

    # 1.) Calculate occupancy and activty
    heatmap_occupancy, xedges, yedges = np.histogram2d(positions[:, 1], positions[:, 0], bins=square_bins, normed=False)
    #print(len(heatmap_occupancy), len(xedges), len(yedges))
    heatmap_activity, xedges, yedges = np.histogram2d(positions[:, 1], positions[:, 0], bins=(xedges, yedges), normed=False, weights=neuron_activation)

    # 2.) Calculate rate map as AVERAGE
    ratemap = heatmap_activity / heatmap_occupancy

    # 2.1) Smooth ratemap TAKE GOOD CARE OF NANS
    # see here (https://stackoverflow.com/questions/18697532/gaussian-filtering-a-image-with-nan-in-python)
    V = ratemap.copy()
    V[ratemap!=ratemap] = 0
    VV = gaussian_filter(V, sigma=smoothing)

    W = 0 * ratemap.copy() + 1
    W[ratemap!=ratemap] = 0
    WW = gaussian_filter(W, sigma=smoothing)

    ratemap_wo_nan = VV / WW
    ratemap_wo_nan[ratemap_wo_nan!=ratemap_wo_nan] = 0

    ratemap_with_nan = ratemap_wo_nan.copy()
    ratemap_with_nan[ratemap!=ratemap] = np.NaN

    # 3.) Return with edges
    #return (heatmap_occupancy, ratemap, ratemap_with_nan, ratemap_wo_nan, xedges, yedges)
    return (heatmap_activity, ratemap, ratemap_with_nan, ratemap_wo_nan, xedges, yedges)

    # extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    # plt.imshow(ratemap.T, extent=extent, origin='lower')
    # plt.show()


In [ ]:
def rateheatMap_ratelinear_overlay(mode,realOrPred,ratemap,heatmap,rateLinear,overlayRatemap,overlayHeatmap):
    mpl.rcParams.update({'font.size': 20})

    if realOrPred == 'real':
        chosenPosition = real_xy
    elif realOrPred == 'pred':
        chosenPosition = pred_xy
    
    lenXY = list(range(0, chosenPosition.shape[0], 1)) # inputs: pred_xy or real_xy
    lenImage = list(range(0, hidden_act.shape[1], 1)) # outputs: dense layer (Neurons number)

    real_xy_X = []
    real_xy_Y = []
    
    for i in lenXY:
        x = chosenPosition[i,0]
        y = chosenPosition[i,1]
        real_xy_X.append(x)
        real_xy_Y.append(y)  
    maxX = max(real_xy_X)
    maxY = max(real_xy_Y)         
    
    positions = chosenPosition
    
    nt = time.strftime("%d-%m-%Y_%H%M")
    dt = time.strftime("%Y-%m-%d_%H.%M.%S")
    
    #if rateOrHeat =='rate':
    if ratemap == 'yes':
        folder_rate = dir13 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_' + nt
        if not os.path.exists(folder_rate):
            os.makedirs(folder_rate)
    
    #elif rateOrHeat =='heat':
    if heatmap == 'yes':
        folder_heat = dir15 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_' + nt
        if not os.path.exists(folder_heat):
            os.makedirs(folder_heat)
    
    if rateLinear == 'yes':
        folder_rl = dir18 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_' + nt # + '_axis=' + axis
        if not os.path.exists(folder_rl):
            os.makedirs(folder_rl)
            
    #if overlayRatemap == 'yes':
    #    folder_ol = dir19 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_' + nt
    #    if not os.path.exists(folder_ol):
    #        os.makedirs(folder_ol)
    
    #if overlayHeatmap == 'yes':
    #    folder_oh = dir20 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_' + nt
    #    if not os.path.exists(folder_oh):
    #        os.makedirs(folder_oh)
    
    rmwon_all = []
    hm_all = []
    rmwon_all_try = np.zeros((25,25)) # same as the num_bins of generate_ratemap_heatmap_from_activations
    hm_all_try = np.zeros((25,25))
    
    for a in lenImage:        
        neuron_activation = hidden_act[:,a]
        #print(neuron_activation.shape)
        #print(positions.shape)
        (hm,rm, rmwn, rmwon, xedge, yedge) = generate_ratemap_heatmap_from_activations(neuron_activation, positions, num_bins=25, smoothing=1, max_pos=(maxX,maxY))
        
        rmwon_all.append(rmwon)
        hm_all.append(hm)
        
        rmwon_all_try += rmwon
        hm_all_try += hm
        
        #if rateOrHeat == 'rate':
        if ratemap == 'yes':
            plt.matshow(rmwon) #, extent=[0,120,1000,0],aspect='auto')
            plt.savefig(folder_rate + '/' + 'ratemap' + str(a) + '_' + dt + '.png')
            
        #elif rateOrHeat =='heat':
        if heatmap == 'yes':
            plt.matshow(hm)
            plt.savefig(folder_heat + '/' + 'heatmap' + str(a) + '_' + dt + '.png')
            
        if rateLinear == 'yes':
            img_rm = rmwon
            
            mean0 = np.mean(img_rm, axis=(0))
            #mean0 = np.around(np.mean(img_rm, axis=(0)), decimals=4) # Do not do this, it messes the precision
            shape0 = mean0.shape

            mean1 = np.mean(img_rm, axis=(1))
            #mean1 = np.around(np.mean(img_rm, axis=(1)), decimals=4) # Do not do this, it messes the precision
            shape1 = mean1.shape
            
            
            allImage = []
            allImage.extend((img_rm,mean0,mean1))

            fig, axes = plt.subplots(1,3, figsize=(20, 8))
            axes = axes.flatten()
            fig.tight_layout(pad=6.0)
            
            plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = True
            plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = False

            for idx, ax in enumerate(axes):
                
                for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                    label.set_fontsize(20)
                
                if idx == 0:
                    #dataname = (f'Node #{a}_ratemap') #'Original Image'
                    dataname = ('Ratemap')
                    img = allImage[idx]
                    #ax.set_title(dataname, pad=10)
                    ax.set_title(dataname, pad=10, fontsize=25, fontweight='bold')
                    #for tick in ax.xaxis.get_major_ticks():
                    #    tick.label.set_fontsize(20)
                    #ax.set_xticklabels(labels)
                    ax.tick_params(axis="x", bottom=True, top=False, labelbottom=False, labeltop=False,which='major', labelsize=5)
                    ax.set_xticks([0,0,5,10,15,20])
                    ax.set_xticklabels(['0','0','5','10','15','20'],fontsize=20)#, y=0)
                    ax.xaxis.set_visible(True)
                    ax.xaxis.set_ticks_position('bottom')
                    ax.xaxis.tick_bottom()
                    ax.matshow(img, aspect='auto')
                elif idx == 1:
                    #dataname = (f'axis=(0)_x-axis, shape={shape0}')
                    dataname = ('X-axis (Index/Row)')
                    img = allImage[idx]
                    #ax.set_title(dataname, pad=10)
                    ax.set_title(dataname, pad=10, fontsize=25, fontweight='bold')
                    ax.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
                    ax.xaxis.set_ticks_position('top')
                    #ax.tick_params(axis='x', labelsize=20)
                    ax.set_xticks([0,0,5,10,15,20])
                    ax.set_xticklabels(['0','0','5','10','15','20'],fontsize=20)
                    ax.plot(img)
                elif idx == 2:
                    #dataname = (f'\n\n\n\n\n\n\n axis=(1)_y-axis, shape={shape1}')
                    dataname = ('\n\n\n\n\n\n\n Y-axis (Column)')
                    img = allImage[idx]
                    #ax.set_title(dataname, pad=10)
                    ax.set_title(dataname, pad=10, fontsize=25, fontweight='bold')
                    ax.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
                    ax.xaxis.set_ticks_position('top')
                    #ax.tick_params(axis='x', labelsize=20)
                    ax.set_xticks([0,0,5,10,15,20])
                    ax.set_xticklabels(['0','0','5','10','15','20'],fontsize=20)
                    ax.plot(img)
                
            fig.savefig(folder_rl + '/' + 'ratelinear' + str(a) + '_' + dt + '.png')
            #fig.clf()
     
    # While primary inspection on the codes evaluate that the overlayRatemap and overlayHeatmap are valid,
    # which I even try with 2 slightly different approaches,
    # visual inspection on the generated data tells otherwise, hence the part below is subject to modify
    # However, there are separate functions (following later) which do the right jobs
    rmwon_all=np.array(rmwon_all)
    hm_all=np.array(hm_all)
    
    if overlayRatemap == 'yes':
        overlay = np.mean(rmwon_all, axis=(0)) 

        fig, ax = plt.subplots()
        cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
        im = ax.imshow(overlay)#, cmap='gist_earth')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        #ax.axis('off')
        fig.colorbar(im, cax=cax, orientation='horizontal')
        endfig = plt.gcf()
        plt.show()

        ##plt.savefig(folder_ol + '/' + 'Overlay' + str(a) + '_' + dt + '.png')
        endfig.savefig(dir19 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_OverlayRatemap_' + dt + '.png')

        ############################# Test Starts #################
        len_rm = rmwon_all_try.shape[0]
        rmwon_all_try /= len_rm
        
        fig, ax = plt.subplots()
        cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
        im = ax.imshow(rmwon_all_try)#, cmap='gist_earth')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        #ax.axis('off')
        fig.colorbar(im, cax=cax, orientation='horizontal')
        endfig = plt.gcf()
        plt.show()

        ##plt.savefig(folder_ol + '/' + 'Overlay' + str(a) + '_' + dt + '.png')
        endfig.savefig(dir19 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_OverlayRatemap_Try_' + dt + '.png')
        ############################# Test Ends #################
        
        
    if overlayHeatmap == 'yes':
        overlay_hm = np.mean(hm_all, axis=(0)) 

        fig, ax = plt.subplots()
        cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
        im = ax.imshow(overlay_hm)#, cmap='gist_earth')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        #ax.axis('off')
        fig.colorbar(im, cax=cax, orientation='horizontal')
        endfig = plt.gcf()
        plt.show()

        ##plt.savefig(folder_oh + '/' + 'Overlay' + str(a) + '_' + dt + '.png')
        endfig.savefig(dir20 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_OverlayHeatmap_' + dt + '.png')
        
        ############################# Test Starts #################
        len_hm = hm_all_try.shape[0]
        hm_all_try /= len_hm
        
        fig, ax = plt.subplots()
        cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
        im = ax.imshow(hm_all_try)#, cmap='gist_earth')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        #ax.axis('off')
        fig.colorbar(im, cax=cax, orientation='horizontal')
        endfig = plt.gcf()
        plt.show()

        ##plt.savefig(folder_ol + '/' + 'Overlay' + str(a) + '_' + dt + '.png')
        endfig.savefig(dir20 + '/' + test_runNum + '-' + mode + '_' + realOrPred + '_xy_OverlayHeatmap_Try_' + dt + '.png')
        ############################# Test Ends #################

In [ ]:
def combineAllRateHeatMaps(mapType,GPU,runType,num,dense): # ratemap = dir13, heatmap = dir15
    
    if mapType == 'ratemap':
        dirNum = dir13
    elif mapType == 'heatmap':
        dirNum = dir15
    
    if dense == 128:
        fig_row = 16
    elif dense == 64:
        fig_row = 8
    elif dense == 32:
        fig_row = 4
    elif dense == 16:
        fig_row = 2
    elif dense == 8:
        fig_row = 1
    
    from pathlib import Path
    import os
    import re
    import glob
    import time
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    
    os.environ["CUDA_VISIBLE_DEVICES"]="f{GPU}" # nothing = CPU, 0 = GeForce GTX 1660, 1 = Quadro K1200

    ## Lines below must be performed for Ubuntu 18.04.3
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)
    
    %run ./CommonFunctions_OthersGeneral.ipynb
    
    def sorted_alphanumeric(data):
        convert = lambda text: int(text) if text.isdigit() else text.lower()
        alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
        return sorted(data, key=alphanum_key)
        
    ## The 4 lines below avtually helps with sorting and combining full path name##
    # The downside of this is that it sort based on folder last modified date and not name
    def sorted_dirModifiedDate(directory):
        files = list(filter(os.path.isdir, glob.glob(directory + "*")))
        files.sort(key=lambda x: os.path.getmtime(x),reverse=False)
        return files
    
    
    if runType == 'all':        
        folderlist = sorted_dirModifiedDate(directory = dirNum)
        numOfFolder = list(range(0, len(folderlist),1))

        for i in numOfFolder:
            folderPath = folderlist[i]
            folderName = folderPath[-15:]
 
            for p in Path(folderPath).glob("combined*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            for p in Path(folderPath).glob("overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            for p in Path(folderPath).glob("Recolored_overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            fig, axes = plt.subplots(fig_row,8, figsize=(10, 10))
            axes = axes.flatten()
            
            for r,d,f in os.walk(folderPath):
                file = sorted_alphanumeric(f)
                for idx, ax in enumerate(axes):
                    file_path = r + '/' + f[idx]
                    img=mpimg.imread(file_path)
                    ax.axis('off')
                    ax.matshow(img ,aspect='auto')      
                fig.savefig(folderPath + '/' + 'combined' + '_' + mapType + '_' + folderName + '.png')
     
    
    elif runType == 'current':
        fullList = []
        folderlist = []

        fullList = sorted_dirModifiedDate(directory = dirNum)

        numOfFolder_len = int(len(fullList)) #equivalent to: numOfFolder_len = sum([len(d) for r,d,f in os.walk(dir13)])

        folderlist = [fullList[numOfFolder_len-4],fullList[numOfFolder_len-3],fullList[numOfFolder_len-2],fullList[numOfFolder_len-1]]

        numOfFolder = list(range(0, len(folderlist),1))

        for i in numOfFolder:
            folderPath = folderlist[i]
            folderName = folderPath[-15:]
            print(folderName)
            

            for p in Path(folderPath).glob("combined*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127

            for p in Path(folderPath).glob("overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            for p in Path(folderPath).glob("Recolored_overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            fig, axes = plt.subplots(fig_row,8, figsize=(10, 10))
            axes = axes.flatten()

            for r,d,f in os.walk(folderPath):
                file = sorted_alphanumeric(f)
                for idx, ax in enumerate(axes):
                    file_path = r + '/' + f[idx]
                    img=mpimg.imread(file_path)
                    ax.axis('off')
                    ax.matshow(img ,aspect='auto')    

                fig.savefig(folderPath + '/' + 'combined' + '_' + mapType + '_' + folderName + '.png')
    
    
    elif runType == 'some_lastfew':
        fullList = []
        folderlist = []
       
        fullList = sorted_dirModifiedDate(directory = dirNum)
        numOfFolder_len = int(num)
        
        numOfFolder = list(range(0,numOfFolder_len,1))
        folderlist = []

        for i in numOfFolder:
            name = fullList[numOfFolder_len - (i+1)]
            folderlist.append(name)


        for i in numOfFolder:
            folderPath = folderlist[i]
            folderName = folderPath[-15:]
            print(folderName)
            
            for p in Path(folderPath).glob("combined*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127

            for p in Path(folderPath).glob("overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            for p in Path(folderPath).glob("Recolored_overlay*.png"):
                p.unlink()   ## delete the previous combination if any so we get the 128 plots and not combined(the 1st subplot) + 127
            
            fig, axes = plt.subplots(fig_row,8, figsize=(10, 10))
            axes = axes.flatten()

            for r,d,f in os.walk(folderPath):
                file = sorted_alphanumeric(f)
                for idx, ax in enumerate(axes):
                    file_path = r + '/' + f[idx]
                    img=mpimg.imread(file_path)
                    ax.axis('off')
                    ax.matshow(img ,aspect='auto')    

                fig.savefig(folderPath + '/' + 'combined' + '_' + mapType + '_' + folderName + '.png')
                #fig.clf()
                ##fig.close('all')

In [ ]:
# Overlay post ratemap. (Can also try to make stacks while generating ratemap when free, or use transparent channel of png to try)

def overlay_PostRateHeatmap_inFolder(GPU,runType,num,mapType):
    
    if mapType == 'ratemap':
        dirNum = dir13
        mapName = 'ratemap'
    elif mapType == 'heatmap':
        dirNum = dir15
        mapName = 'heatmap'
    
    from pathlib import Path
    import os
    import fnmatch
    import re
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np
    import glob
    
    os.environ["CUDA_VISIBLE_DEVICES"]="f{GPU}" # nothing = CPU, 0 = GeForce GTX 1660, 1 = Quadro K1200

    ## Lines below must be performed for Ubuntu 18.04.3
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

    %run ./CommonFunctions_OthersGeneral.ipynb
  
    ## The 4 lines below avtually helps with sorting and combining full path name##
    # The downside of this is that it sort based on folder last modified date and not name
    def sorted_dirModifiedDate(directory):
        files = list(filter(os.path.isdir, glob.glob(directory + "*")))
        files.sort(key=lambda x: os.path.getmtime(x),reverse=False)
        return files


    if runType == 'all':
        foldername = []
        folderlist = []
 
        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
            

        index=0
        for i in folderlist:

            for p in Path(i).glob("overlay*.png"):
                p.unlink()

            all_images = []
            files = []
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)

            for row in files:
                img = mpimg.imread(row)
                image = img
                all_images.append(image)

            combi = np.sum(all_images, axis=0)
            overlay = combi/totalImages

            fig, ax = plt.subplots()
            cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
            im = ax.imshow(overlay)#, cmap='gist_earth')
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            #ax.axis('off')
            fig.colorbar(im, cax=cax, orientation='horizontal')
            plt.show()

            fig.savefig(i + '/' + 'overlay_folder' + foldername[index] + '.png')
            index+=1         
            
        
    elif runType == 'current':
        foldername = []
        folderlist = []
 
        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
        
        numOfFolder_len = int(len(folderlist)) #equivalent to: numOfFolder_len = sum([len(d) for r,d,f in os.walk(dir13)])
        
        folderIndex = [[numOfFolder_len-4],[numOfFolder_len-3],[numOfFolder_len-2],[numOfFolder_len-1]]
        newfolderlist = [folderlist[numOfFolder_len-4],folderlist[numOfFolder_len-3],folderlist[numOfFolder_len-2],folderlist[numOfFolder_len-1]]
        folderlist = newfolderlist
        numOfFolder = list(range(0, len(folderlist),1))
        
        index=0
        for i in folderlist:

            for p in Path(i).glob("overlay*.png"):
                p.unlink()

            all_images = []
            files = []
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)

            for row in files:
                img = mpimg.imread(row)
                image = img
                all_images.append(image)

            combi = np.sum(all_images, axis=0)
            overlay = combi/totalImages

            fig, ax = plt.subplots()
            cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
            im = ax.imshow(overlay)#, cmap='gist_earth')
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            #ax.axis('off')
            fig.colorbar(im, cax=cax, orientation='horizontal')
            plt.show()

            fig.savefig(i + '/' + 'overlay_folder' + foldername[folderIndex[index][0]] + '.png')
            index+=1
            
            
    elif runType == 'some_lastfew':
        foldername = []
        folderlist = []
        newfolderlist = []
        newfoldername = []

        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
        
        numOfFolder_len = int(num)

        numOfFolder = list(range(0,numOfFolder_len,1))
        
        for i in numOfFolder:
            name = folderlist[len(folderlist) - (i+1)]
            fname = foldername[len(foldername) - (i+1)]
            newfolderlist.append(name)
            newfoldername.append(fname)
            
        folderlist = newfolderlist
        foldername = newfoldername
        
        index=0
        for i in folderlist:

            for p in Path(i).glob("overlay*.png"):
                p.unlink()

            all_images = []
            files = []
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)

            for row in files:
                img = mpimg.imread(row)
                image = img
                all_images.append(image)

            combi = np.sum(all_images, axis=0)
            overlay = combi/totalImages

            fig, ax = plt.subplots()
            cax = fig.add_axes([0.31, 0.96, 0.4, 0.02]) #cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
            im = ax.imshow(overlay)#, cmap='gist_earth')
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            #ax.axis('off')
            fig.colorbar(im, cax=cax, orientation='horizontal')
            plt.show()

            fig.savefig(i + '/' + 'overlay_folder' + foldername[index] + '.png')
            index+=1
            

In [ ]:
# Change the overlay into matching colors of neuron activations (ratemaps) - into greyscale and back into gist_earth
# Not very accurate as the activation bar may show a 4 times lower range than the actual activated values,
# (the conversion is correct though)
# but this can be altered, subject to adjustment to the bar value or colors (need further examine)
def overlayGreyScaling_PostRateHeatmapOverlay_inFolder(GPU,runType,num,mapType):
    
    if mapType == 'ratemap':
        dirNum = dir13
        mapName = 'ratemap'
    elif mapType == 'heatmap':
        dirNum = dir15
        mapName = 'heatmap'
    
    from pathlib import Path
    import os
    import fnmatch
    import re
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np
    from pathlib import Path
    import glob
    import time
    
    os.environ["CUDA_VISIBLE_DEVICES"]="f{GPU}" # nothing = CPU, 0 = GeForce GTX 1660, 1 = Quadro K1200

    ## Lines below must be performed for Ubuntu 18.04.3
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

    %run ./CommonFunctions_OthersGeneral.ipynb
  
    ## The 4 lines below avtually helps with sorting and combining full path name##
    # The downside of this is that it sort based on folder last modified date and not name
    def sorted_dirModifiedDate(directory):
        files = list(filter(os.path.isdir, glob.glob(directory + "*")))
        files.sort(key=lambda x: os.path.getmtime(x),reverse=False)
        return files


    if runType == 'all':
        foldername = []
        folderlist = []
        #Overallratemap = np.zeros((288,288,4))
 
        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
            

        index=0
        for i in folderlist:

            for p in Path(i).glob("Recolored_overlay*.png"):
                p.unlink()
                
            for f in os.listdir(i):
                #if fnmatch.fnmatch(f, 'overlay*.png'):
                if fnmatch.fnmatch(f, f'{mapName}0*.png'):
                    ImageSample = mpimg.imread((i+'/'+f))
                    #ImageSample = ImageSample.shape
                    #print(ImageSample)
                    
            Overallratemap = np.zeros(ImageSample.shape)

            all_images = []
            files = []
            
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)
            
            for row in files:
                img = mpimg.imread(row)
                Overallratemap += img

            Overallratemap /= totalImages
            #print(Overallratemap.shape)


            fig, ax = plt.subplots()
            grey = Overallratemap[:,:,0]
            #cax = fig.add_axes([0.31, 0.96, 0.4, 0.02])
            test = ax.imshow(grey,cmap = 'gist_earth')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            #plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
            ##plt.margins(0,0)
            ##fig.colorbar(test, cax=cax, orientation='horizontal')
            fig.colorbar(test, orientation='vertical')
            plt.show()

            fig.savefig(i + '/' + 'Recolored_overlay_folder' + foldername[index] + '.png')
            index+=1         
            
        
    elif runType == 'current':
        foldername = []
        folderlist = []
        #Overallratemap = np.zeros((288,288,4))
 
        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
        
        numOfFolder_len = int(len(folderlist)) #equivalent to: numOfFolder_len = sum([len(d) for r,d,f in os.walk(dir13)])
        
        folderIndex = [[numOfFolder_len-4],[numOfFolder_len-3],[numOfFolder_len-2],[numOfFolder_len-1]]
        newfolderlist = [folderlist[numOfFolder_len-4],folderlist[numOfFolder_len-3],folderlist[numOfFolder_len-2],folderlist[numOfFolder_len-1]]
        folderlist = newfolderlist
        numOfFolder = list(range(0, len(folderlist),1))
        
        index=0
        for i in folderlist:

            for p in Path(i).glob("Recolored_overlay*.png"):
                p.unlink()
                
            for f in os.listdir(i):
                #if fnmatch.fnmatch(f, 'overlay*.png'):
                if fnmatch.fnmatch(f, f'{mapName}0*.png'):
                    ImageSample = mpimg.imread((i+'/'+f))
                    #ImageSample = ImageSample.shape
                    #print(ImageSample)
                    
            Overallratemap = np.zeros(ImageSample.shape)

            all_images = []
            files = []
            
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)

            for row in files:
                img = mpimg.imread(row)
                Overallratemap += img

            Overallratemap /= totalImages
            #print(Overallratemap.shape)


            fig, ax = plt.subplots()
            grey = Overallratemap[:,:,0]
            #cax = fig.add_axes([0.31, 0.96, 0.4, 0.02])
            test = ax.imshow(grey,cmap = 'gist_earth')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            #plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
            ##plt.margins(0,0)
            ##fig.colorbar(test, cax=cax, orientation='horizontal')
            fig.colorbar(test, orientation='vertical')
            plt.show()

            fig.savefig(i + '/' + 'Recolored_overlay_folder' + foldername[folderIndex[index][0]] + '.png')
            index+=1
            
            
    elif runType == 'some_lastfew':
        foldername = []
        folderlist = []
        newfolderlist = []
        newfoldername = []
        #Overallratemap = np.zeros((288,288,4))

        folderlist = sorted_dirModifiedDate(directory = dirNum)
        for i in folderlist:
            folderName = i[-15:]
            foldername.append(folderName)
        
        numOfFolder_len = int(num)

        numOfFolder = list(range(0,numOfFolder_len,1))
        
        for i in numOfFolder:
            name = folderlist[len(folderlist) - (i+1)]
            fname = foldername[len(foldername) - (i+1)]
            newfolderlist.append(name)
            newfoldername.append(fname)
            
        folderlist = newfolderlist
        foldername = newfoldername
        
        index=0
        for i in folderlist:

            for p in Path(i).glob("Recolored_overlay*.png"):
                p.unlink()
                
            for f in os.listdir(i):
                #if fnmatch.fnmatch(f, 'overlay*.png'):
                if fnmatch.fnmatch(f, f'{mapName}0*.png'):
                    ImageSample = mpimg.imread((i+'/'+f))
                    #ImageSample = ImageSample.shape
                    #print(ImageSample)
                    
            Overallratemap = np.zeros(ImageSample.shape)

            all_images = []
            files = []
            
            for f in os.listdir(i):
                if fnmatch.fnmatch(f, f'{mapName}*.png'):
                    path = ((i+'/'+f))
                    files.append(path)        
            totalImages = len(files)

            for row in files:
                img = mpimg.imread(row)
                Overallratemap += img

            Overallratemap /= totalImages
            #print(Overallratemap.shape)


            fig, ax = plt.subplots()
            grey = Overallratemap[:,:,0]
            #cax = fig.add_axes([0.31, 0.96, 0.4, 0.02])
            test = ax.imshow(grey,cmap = 'gist_earth')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            #plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
            ##plt.margins(0,0)
            ##fig.colorbar(test, cax=cax, orientation='horizontal')
            fig.colorbar(test, orientation='vertical')
            plt.show()

            fig.savefig(i + '/' + 'Recolored_overlay_folder' + foldername[index] + '.png')
            index+=1
            

In [ ]:
# Full Guides
# Without Angles

##def allPlot_NoAngles():

#plot_history(angles='no')
plot_history(angles='no',lr='no')
plt.pause(5) #plt.close()

################ TRAINING #################
real_xy, pred_xy, real_img = activationModelSelection_Angles(angles='no',mode='training',times=125)

layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
activation_model = Model(inputs=model.input,outputs=layer_outputs)
hidden_act = activation_model.predict(real_img)

plt.pause(10)

AnglesPredictionPlot(angles='no',mode='training')

rateheatMap_ratelinear_overlay(mode='training',realOrPred='real',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='training',realOrPred='pred',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')

################# TESTING #################
real_xy, pred_xy, real_img = activationModelSelection_Angles(angles='no',mode='testing',times=125)

layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
activation_model = Model(inputs=model.input,outputs=layer_outputs)
hidden_act = activation_model.predict(real_img)

plt.pause(10)

AnglesPredictionPlot(angles='no',mode='testing')

# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='testing',realOrPred='real',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='testing',realOrPred='pred',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')

################# For Both #################

combineAllRateHeatMaps(mapType='ratemap',GPU=0,runType='current',num='',dense=dense)
combineAllRateHeatMaps(mapType='heatmap',GPU=0,runType='current',num='',dense=dense)
overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='ratemap')
overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='heatmap')
overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='ratemap',GPU=0,runType='current',num='')
overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='heatmap',GPU=0,runType='current',num='')

# Full Guides
# With Angles

#def allPlot_Angles():

#plot_history(angles='yes')
plot_history(angles='yes',lr='no')
plt.pause(5) #plt.close()

################# TRAINING #################
real_xy, pred_xy, real_angles, pred_angles, real_img = activationModelSelection_Angles(angles='yes',
                                                                                       mode='training',times=125)

layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
activation_model = Model(inputs=model.input,outputs=layer_outputs)
hidden_act = activation_model.predict(real_img)

plt.pause(10)

AnglesPredictionPlot(angles='yes',mode='training')

rateheatMap_ratelinear_overlay(mode='training',realOrPred='real',ratemap='yes',heatmap='yes',rateLinear='yes',
                               overlayRatemap='yes',overlayHeatmap='yes')
# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='training',realOrPred='pred',ratemap='yes',heatmap='yes',rateLinear='yes',
                               overlayRatemap='yes',overlayHeatmap='yes')

################# TESTING #################
real_xy, pred_xy, real_angles, pred_angles, real_img = activationModelSelection_Angles(angles='yes',
                                                                                       mode='testing',times=125)

layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
activation_model = Model(inputs=model.input,outputs=layer_outputs)
hidden_act = activation_model.predict(real_img)

plt.pause(10)

AnglesPredictionPlot(angles='yes',mode='testing')

# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='testing',realOrPred='real',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
# time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
rateheatMap_ratelinear_overlay(mode='testing',realOrPred='pred',ratemap='yes',heatmap='yes',
                               rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')

################# For Both #################

combineAllRateHeatMaps(mapType='ratemap',GPU=0,runType='current',num='',dense=128)
combineAllRateHeatMaps(mapType='heatmap',GPU=0,runType='current',num='',dense=128)
overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='ratemap')
overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='heatmap')
overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='ratemap',GPU=0,runType='current',num='')
overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='heatmap',GPU=0,runType='current',num='')

# Full Guides
# Combined all steps into one from the 2 below: Without Angles or With Angles
# Not using, for unknown reason, keep encountering the error <name 'real_xy' is not defined> when scripts are correct

def combine_RunAllSteps(angles):
    if angles == 'yes':
        allPlot_Angles()
    elif angles == 'no':
        allPlot_NoAngles()

#def combine_RunAllSteps(angles): # Not using, for unknown reason, keep encountering the error <name 'real_xy' is not defined> when scripts are correct
#    if angles == 'yes':
#        #plot_history(angles='yes')
#        plot_history(angles='yes',lr='no')
#        plt.pause(5) #plt.close()
#
#        ################# TRAINING #################
#        real_xy, pred_xy, real_angles, pred_angles, real_img = activationModelSelection_Angles(angles='yes',
#                                                                                               mode='training',
#                                                                                               times=125)
#        plt.pause(10) #plt.close()
#        
#        layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
#        activation_model = Model(inputs=model.input,outputs=layer_outputs)
#        hidden_act = activation_model.predict(real_img)
#
#        AnglesPredictionPlot(angles='yes',mode='training')
#        #plt.pause(10) #plt.close()
#
#        rateheatMap_ratelinear_overlay(mode='training',realOrPred='real',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='training',realOrPred='pred',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#
#        ################# TESTING #################
#        real_xy, pred_xy, real_angles, pred_angles, real_img = activationModelSelection_Angles(angles='yes',
#                                                                                               mode='testing',
#                                                                                               times=125)
#        plt.pause(10) #plt.close()
#        
#        layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
#        activation_model = Model(inputs=model.input,outputs=layer_outputs)
#        hidden_act = activation_model.predict(real_img)
#
#        AnglesPredictionPlot(angles='yes',mode='testing')
#        #plt.pause(10) #plt.close()
#
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='testing',realOrPred='real',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='testing',realOrPred='pred',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#
#        ################# For Both #################
#
#        combineAllRateHeatMaps(mapType='ratemap',GPU=0,runType='current',num='',dense=128)
#        combineAllRateHeatMaps(mapType='heatmap',GPU=0,runType='current',num='',dense=128)
#        overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='ratemap')
#        overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='heatmap')
#        overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='ratemap',GPU=0,runType='current',num='')
#        overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='heatmap',GPU=0,runType='current',num='')
#
#    elif angles == 'no':
#        #plot_history(angles='no')
#        plot_history(angles='no',lr='no')
#        plt.pause(5) #plt.close()
#
#        ################# TRAINING #################
#        real_xy, pred_xy, real_img = activationModelSelection_Angles(angles='no',mode='training',times=125)
#        #print(f'real_xy = {real_xy}')
#        #print(f'pred_xy = {pred_xy}')
#        plt.pause(10) #plt.close()
#
#        layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
#        activation_model = Model(inputs=model.input,outputs=layer_outputs)
#        hidden_act = activation_model.predict(real_img)
#
#        AnglesPredictionPlot(angles='no',mode='training')
#        #plt.pause(10) #plt.close()
#
#        rateheatMap_ratelinear_overlay(mode='training',realOrPred='real',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='training',realOrPred='pred',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#
#        ################# TESTING #################
#        real_xy, pred_xy, real_img = activationModelSelection_Angles(angles='no',mode='testing',times=125)
#        plt.pause(10) #plt.close()
#
#        layer_outputs = model.layers[27].output #[layer.output for layer in model.layers[27]]
#        activation_model = Model(inputs=model.input,outputs=layer_outputs)
#        hidden_act = activation_model.predict(real_img)
#
#        AnglesPredictionPlot(angles='no',mode='testing')
#        #plt.pause(10) #plt.close()
#
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='testing',realOrPred='real',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#        # time.sleep(60) Not needed anymore because the folder name has indicated the mode now, so will not have same name again
#        rateheatMap_ratelinear_overlay(mode='testing',realOrPred='pred',ratemap='yes',heatmap='yes',
#                                       rateLinear='yes',overlayRatemap='yes',overlayHeatmap='yes')
#
#        ################# For Both #################
#
#        combineAllRateHeatMaps(mapType='ratemap',GPU=0,runType='current',num='',dense=128)
#        combineAllRateHeatMaps(mapType='heatmap',GPU=0,runType='current',num='',dense=128)
#        overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='ratemap')
#        overlay_PostRateHeatmap_inFolder(GPU=0,runType='current',num='',mapType='heatmap')
#        overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='ratemap',GPU=0,runType='current',num='')
#        overlayGreyScaling_PostRateHeatmapOverlay_inFolder(mapType='heatmap',GPU=0,runType='current',num='')
#        
#
#combine_RunAllSteps(angles='no')